In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath('../..'))
from experiments.iter1.util.importer import *
uk_path = "../../src/data_preprocess/dataset/UKDATA_cleaned.csv"
nist_path = "../../src/data_preprocess/dataset/NIST_cleaned.csv"

from experiments.iter1.util.evaluate import evaluate_model, plot_models

assert time_horizon > 0, "Time horizon must be a positive integer"

# UK

In [ ]:
df = pd.read_csv(uk_path)

cleaner = TempCleaner(clean_pow_low, clean_in_low, clean_in_high, clean_out_low, clean_out_high, clean_delta_temp)
splitter = DaySplitter(TIMESTAMP, POWER, train_days, val_days, test_days) 

uk_model = NaiveProbabilisticBaseline.Builder() \
    .add_data(df) \
    .set_cleaner(cleaner) \
    .set_normalizer_class(MinMaxNormalizer) \
    .set_splitter(splitter) \
    .set_sequencer_class(AllTimeSequencer) \
    .set_target_column(TARGET_COLUMN) \
    .set_power_column(POWER_COLUMN) \
    .set_outdoor_column(OUTDOOR_COLUMN) \
    .set_batch_size(batch_size) \
    .set_horizon_len(time_horizon) \
    .set_worker_num(NUM_WORKERS) \
    .set_seq_len(time_horizon) \
    .set_error(NRMSE) \
    .set_train_error(RMSE) \
    .add_test_error(NMCRPS) \
    .set_penalty_strat(Naive) \
    .add_test_error(CALE) \
    .build()

#uk_test_results = uk_model.test()
#
#uk_model.reset_forward_memory()
#
#uk_naive_flex_results = evaluate_model(uk_model, df, splitter, cleaner, TIMESTAMP, POWER, on_limit_w, off_limit_w, consecutive_points, time_horizon, time_horizon, TARGET_COLUMN, error, temp_boundary, 0.95)
#
#uk_naive_results = {**uk_test_results, **uk_naive_flex_results}
#uk_naive_results["title"] = "Naive"

In [ ]:
best_total_mafe = 1.030161991114778
best_confidence = 0.20

# UK PLOT

In [ ]:


uk_predictions = uk_model.get_predictions()


plot_models(
    [uk_predictions],
    time_horizon,
    uk_model.get_timestamps(),
    uk_model.get_actuals(),
    titels=["UK Dataset"])

 # NIST

In [ ]:
df = pd.read_csv(nist_path)

cleaner = TempCleaner(clean_pow_low, clean_in_low, clean_in_high, clean_out_low, clean_out_high, clean_delta_temp)
splitter = DaySplitter(TIMESTAMP, POWER, train_days, val_days, test_days)

nist_naive_model = NaiveProbabilisticBaseline.Builder() \
    .add_data(df) \
    .set_cleaner(cleaner) \
    .set_normalizer_class(MinMaxNormalizer) \
    .set_splitter(splitter) \
    .set_sequencer_class(AllTimeSequencer) \
    .set_target_column(TARGET_COLUMN) \
    .set_power_column(POWER_COLUMN) \
    .set_outdoor_column(OUTDOOR_COLUMN) \
    .set_horizon_len(time_horizon) \
    .set_worker_num(NUM_WORKERS) \
    .set_seq_len(time_horizon) \
    .set_error(NRMSE) \
    .set_train_error(RMSE) \
    .add_test_error(NMCRPS) \
    .set_penalty_strat(Naive) \
    .add_test_error(CALE) \
    .build()

nist_naive_test_results = nist_naive_model.test()

nist_naive_model.reset_forward_memory()

nist_naive_flex_results = evaluate_model(nist_naive_model, df, splitter, cleaner, TIMESTAMP, POWER, on_limit_w, off_limit_w, consecutive_points, time_horizon, time_horizon, TARGET_COLUMN, error, temp_boundary, 0.95)
nist_naive_results = {**nist_naive_test_results, **nist_naive_flex_results}
nist_naive_results["title"] = "Naive"

# NIST PLOT

In [ ]:
nist_predictions = nist_naive_model.get_predictions()


plot_models(
    [nist_predictions],
    time_horizon,
    nist_naive_model.get_timestamps(),
    nist_naive_model.get_actuals(),
    titels=["NIST Dataset"])